In [19]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
from scipy.stats.contingency import odds_ratio

# Путь к файлу
file_path = 'ЭР.xlsm'

# Чтение данных из файла Excel
try:
    # Попытка загрузить файл
    genetic_data = pd.read_excel(file_path)
except Exception as e:
    # В случае ошибки при чтении файла, сохранить сообщение об ошибке
    error_message = str(e)
    genetic_data = None

# Вывод первых нескольких строк данных для проверки содержимого или сообщения об ошибке
genetic_data.head() if genetic_data is not None else error_message

,№ п\п,Диагноз,Вегетативно-сосудистая_полинейропатия,Пояснично-крестцовая радикулопатия,Периферический ангиодистонический синдром,Шейно-плечевая радикулопатия,Тяжесть заболевание,Гипертоническая болезнь,Ишемическая болезнь,ЦВЗ,...,rs4880_T,rs4880_C,rs361525_G,rs361525_A,rs16944_A,rs16944_G,rs1799750_G,rs1799750_del,rs1800795_C,rs1800795_G
0,1,VibrationDisease,True,False,False,False,0,True,False,False,...,1,0,1,1,1,1,0,1,1,0
1,2,VibrationDisease,False,False,False,False,2,True,False,False,...,1,1,1,0,1,1,0,1,1,0
2,3,VibrationDisease,False,False,False,False,1,False,False,False,...,0,1,1,0,0,1,0,1,1,0
3,4,VibrationDisease,False,False,True,False,2,True,True,False,...,1,1,1,0,1,0,1,1,1,0
4,5,VibrationDisease,True,False,True,False,1,True,False,False,...,1,0,1,0,1,0,1,1,1,0


In [20]:
# Задаем соответствия для каждого столбца
mapping_rs4880 = {1: 'CC', 2: 'TC', 3: 'TT'}
mapping_rs361525 = {1: 'AA', 2: 'GA', 3: 'GG'}
mapping_rs16944 = {1: 'AA', 2: 'GA', 3: 'GG'}
mapping_rs1799750 = {1: '2G/2G', 2: '1G/2G', 3: '1G/1G'}

# Применяем соответствия к данным
genetic_data['rs4880'] = genetic_data['rs4880'].map(mapping_rs4880)
genetic_data['rs361525'] = genetic_data['rs361525'].map(mapping_rs361525)
genetic_data['rs16944'] = genetic_data['rs16944'].map(mapping_rs16944)
genetic_data['rs1799750'] = genetic_data['rs1799750'].map(mapping_rs1799750)

# Проверяем результат преобразования, выводя первые несколько строк обновленного датафрейма
genetic_data[['rs4880', 'rs361525', 'rs16944', 'rs1799750']].head()


,rs4880,rs361525,rs16944,rs1799750
0,TT,GA,GA,2G/2G
1,TC,GG,GA,2G/2G
2,CC,GG,GG,2G/2G
3,TC,GG,AA,1G/2G
4,TT,GG,AA,1G/2G


In [21]:
def odds_ratio_ci(table, alpha=0.05):
    # Расчет отношения шансов
    or_value = (table[0, 0] * table[1, 1]) / (table[0, 1] * table[1, 0])
    
    # Расчет стандартной ошибки логарифма отношения шансов
    se_log_or = np.sqrt(1/table[0, 0] + 1/table[0, 1] + 1/table[1, 0] + 1/table[1, 1])
    
    # Доверительный интервал для логарифма отношения шансов
    ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
    
    # Возврат к оригинальному масштабу отношения шансов
    ci_or = [np.exp(ci_log_or[0]), np.exp(ci_log_or[1])]
    
    return or_value, ci_or

In [25]:
# Функция для расчета расширенной таблицы сопряженности, включая абсолютные и процентные значения
def calculate_extended_statistics(genetic_data, genotype_col, feature_col):
    # Получаем уникальные генотипы
    genotypes = genetic_data[genotype_col].unique()
    # Общее количество случаев
    total_count = genetic_data.shape[0]
    # Результаты для каждого генотипа
    results = []

    for genotype in genotypes:
        # Таблица сопряженности для данного генотипа
        feature_positive = genetic_data[genetic_data[feature_col] == True][genotype_col].value_counts()
        feature_negative = genetic_data[genetic_data[feature_col] == False][genotype_col].value_counts()
        
        # Количество случаев с признаком и без него для данного генотипа
        count_positive = feature_positive.get(genotype, 0)
        count_negative = feature_negative.get(genotype, 0)
        
        # Проценты для каждого генотипа
        percent_positive = (count_positive / total_count) * 100
        percent_negative = (count_negative / total_count) * 100
        
        # Создание таблицы сопряженности
        contingency_table = np.array([[count_positive, count_negative],
                                      [feature_positive.sum() - count_positive, 
                                       feature_negative.sum() - count_negative]])
        
        # Расчет Хи-квадрат и p-значения
        chi2, p, _, _ = chi2_contingency(contingency_table, correction=True)
        
        # Расчет OR и 95% CI
        or_value, ci_or = odds_ratio_ci(contingency_table)
        
        # Добавление результатов в список
        results.append({
            'Genotype': genotype,
            'With_Feature': count_positive,
            'With_Feature_Percent': percent_positive,
            'Without_Feature': count_negative,
            'Without_Feature_Percent': percent_negative,
            'Chi2': chi2,
            'p-value': p,
            'Odds Ratio': or_value,
            '95% CI': ci_or
        })
    
    # Создание DataFrame из результатов
    results_df = pd.DataFrame(results)
    
    return results_df

# Расчет расширенной статистики для каждого генотипа и признака "Вегетативно-сосудистая_полинейропатия"
extended_statistics = calculate_extended_statistics(genetic_data, 'rs4880', 'Вегетативно-сосудистая_полинейропатия')

extended_statistics


,Genotype,With_Feature,With_Feature_Percent,Without_Feature,Without_Feature_Percent,Chi2,p-value,Odds Ratio,95% CI
0,TT,7,10.447761,22,32.835821,0.026212,0.871385,0.780992,"[0.2593875311760328, 2.351493490114805]"
1,TC,10,14.925373,19,28.358209,0.903724,0.341785,1.973684,"[0.6616113932264528, 5.887790631724474]"
2,CC,1,1.492537,8,11.940299,0.550410,0.458150,0.301471,"[0.03496209285533841, 2.5995158798697875]"


In [26]:
# Список всех признаков для анализа
features = [
    'Вегетативно-сосудистая_полинейропатия', 'Пояснично-крестцовая радикулопатия', 
    'Периферический ангиодистонический синдром', 'Шейно-плечевая радикулопатия', 
    'Гипертоническая болезнь', 'Ишемическая болезнь', 'ЦВЗ', 
    # 'Остеоартроз', 
    'БА', 'ХОБЛ', 'Нейросенсорная тугоухость', 'СД', 'Пептическая язва'
]

# Список генов для анализа
genes = ['rs4880', 'rs361525', 'rs16944', 'rs1799750']

# Результаты по всем признакам и генам
all_features_results = {}

# Перебираем каждый ген и признак, вычисляя статистику и сохраняя в словаре
for gene in genes:
    gene_results = {}
    for feature in features:
        feature_statistics = calculate_extended_statistics(genetic_data, gene, feature)
        gene_results[feature] = feature_statistics
    all_features_results[gene] = gene_results

# Возвращаем результаты в виде словаря
all_features_results

# Из-за ограничений на количество выводимых данных, мы покажем результат только для первого признака и первого гена
first_gene = genes[0]
first_feature = features[0]
all_features_results[first_gene][first_feature]


C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_12296\474194150.py:6: RuntimeWarning: divide by zero encountered in longlong_scalars
  se_log_or = np.sqrt(1/table[0, 0] + 1/table[0, 1] + 1/table[1, 0] + 1/table[1, 1])
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_12296\474194150.py:9: RuntimeWarning: divide by zero encountered in log
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_12296\474194150.py:9: RuntimeWarning: invalid value encountered in double_scalars
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_12296\474194150.py:3: RuntimeWarning: divide by zero encountered in longlong_scalars
  or_value = (table[0, 0] * table[1, 1]) / (table[0, 1] * table[1, 0])
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_12296\474194150.py:6: RuntimeWarning: divide by zero encountered in longlong_scalars
  se_log_or = np.

,Genotype,With_Feature,With_Feature_Percent,Without_Feature,Without_Feature_Percent,Chi2,p-value,Odds Ratio,95% CI
0,TT,7,10.447761,22,32.835821,0.026212,0.871385,0.780992,"[0.2593875311760328, 2.351493490114805]"
1,TC,10,14.925373,19,28.358209,0.903724,0.341785,1.973684,"[0.6616113932264528, 5.887790631724474]"
2,CC,1,1.492537,8,11.940299,0.550410,0.458150,0.301471,"[0.03496209285533841, 2.5995158798697875]"


In [31]:
# Функция для формирования текстового описания статистических данных
def create_description(statistics_df, feature_name):
    descriptions = []
    
    for _, row in statistics_df.iterrows():
        genotype = row['Genotype']
        with_feature = row['With_Feature']
        without_feature = row['Without_Feature']
        with_feature_percent = row['With_Feature_Percent']
        without_feature_percent = row['Without_Feature_Percent']
        p_value = row['p-value']
        
        # Формируем текст описания
        description = (
            f"В группе больных с признаком {feature_name} генотип '{genotype}' "
            f"встречается с частотой {with_feature_percent:.2f}%, что "
        )
        
        # Добавляем интерпретацию p-значения
        if p_value < 0.05:
            description += (
                f"существенно чаще (p={p_value:.3f}), чем в группе без признака, "
                f"где этот генотип встречается с частотой {without_feature_percent:.2f}%."
            )
        else:
            description += (
                f"не существенно отличается (p={p_value:.3f}) от группы без признака, "
                f"где этот генотип встречается с частотой {without_feature_percent:.2f}%."
            )
        
        descriptions.append(description)
    
    return " ".join(descriptions)

# Демонстрация работы функции на примере первого генотипа и первого признака
sample_description = create_description(extended_statistics, first_feature)
sample_description


"В группе больных с признаком Вегетативно-сосудистая_полинейропатия генотип 'TT' встречается с частотой 10.45%, что не существенно отличается (p=0.871) от группы без признака, где этот генотип встречается с частотой 32.84%. В группе больных с признаком Вегетативно-сосудистая_полинейропатия генотип 'TC' встречается с частотой 14.93%, что не существенно отличается (p=0.342) от группы без признака, где этот генотип встречается с частотой 28.36%. В группе больных с признаком Вегетативно-сосудистая_полинейропатия генотип 'CC' встречается с частотой 1.49%, что не существенно отличается (p=0.458) от группы без признака, где этот генотип встречается с частотой 11.94%."